In [1]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [2]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

For Reference: Units Set: MPa, mm2, mm, N

In [3]:
YoungsModulus = 210000
YM_Unit = "MPa"

Area = 200
A_Unit = "mm2"


# node connections for each element
nArray = np.array([[1,2], [2,3], [3,7], [4,5], [5,6], [6,7], [1,4], [2,5], [3,6], [1,5], [2,6]])
print(f"number of elements: {len(nArray)}")

coords = np.array([[0,0], [4,0], [7,0], [0,2.5], [4,4], [7,4], [8.5,3]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths



# list of angles relative to positive x axis for each element
#aArray = np.array([(7/6), 1, 0.5, (5/6), 1, (5/6), 0.5])
#aArray *= m.pi

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v]
# for forces: [node#, Fx, Fy]

uv_BCs = np.array([[3, 0, 0],[4, 0, None]])

force_BCs = np.array([[4, 0, 3000], [5, 0, -2000], [6, 0, 2000], [7, 0, -5000]])

number of elements: 11


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [4]:
print(range(len(coords)))


range(0, 7)


In [5]:
E = Input("Young's Modulus", YoungsModulus, YM_Unit)

A = Input("Area", Area, A_Unit)

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()



nodeCoordinates = Input("Node Coordinates (x, y)", coords, "None")
nodeCoordinates.printValues()



name: Node numbers
value: [[1 2]
 [2 3]
 [3 7]
 [4 5]
 [5 6]
 [6 7]
 [1 4]
 [2 5]
 [3 6]
 [1 5]
 [2 6]]
unit: None


name: Node Coordinates (x, y)
value: [[0.  0. ]
 [4.  0. ]
 [7.  0. ]
 [0.  2.5]
 [4.  4. ]
 [7.  4. ]
 [8.5 3. ]]
unit: None


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [6]:
numElems = len(nodes.value)
print(f"number of elements: {numElems}")
numNodes = nodes.value.max()
print(f"number of nodes: {numNodes}")

number of elements: 11
number of nodes: 7


In [7]:
length = np.zeros(numElems)
Length_unit = "mm"
print(length)

angle = np.zeros(numElems)
print(len(angle))

for i in range(numElems):
    connection = nodes.value[i]
    N1 = connection[0]
    N2 = connection[1]
    x1 = (nodeCoordinates.value[N1-1])[0]
    y1 = (nodeCoordinates.value[N1-1])[1]

    x2 = (nodeCoordinates.value[N2-1])[0]
    y2 = (nodeCoordinates.value[N2-1])[1]
    length[i] = getTrussLength(x1, y1, x2, y2)
    opposite = y2 - y1
    adjacent = x2 - x1
    angle[i] = m.atan(opposite/adjacent)
    
    print(f"element {i+1}: length = {length[i]} m; angle = {angle[i]}")

length *= 1000
print(length)

print(angle)

lengths = Input("Lengths", length, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", angle, Angle_unit)
angles.printValues()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
11
element 1: length = 4.0 m; angle = 0.0
element 2: length = 3.0 m; angle = 0.0
element 3: length = 3.3541019662496847 m; angle = 1.1071487177940904
element 4: length = 4.272001872658765 m; angle = 0.3587706702705722
element 5: length = 3.0 m; angle = 0.0
element 6: length = 1.8027756377319946 m; angle = -0.5880026035475675
element 7: length = 2.5 m; angle = 1.5707963267948966
element 8: length = 4.0 m; angle = 1.5707963267948966
element 9: length = 4.0 m; angle = 1.5707963267948966
element 10: length = 5.656854249492381 m; angle = 0.7853981633974483
element 11: length = 5.0 m; angle = 0.9272952180016122
[4000.         3000.         3354.10196625 4272.00187266 3000.
 1802.77563773 2500.         4000.         4000.         5656.85424949
 5000.        ]
[ 0.          0.          1.10714872  0.35877067  0.         -0.5880026
  1.57079633  1.57079633  1.57079633  0.78539816  0.92729522]


name: Lengths
value: [4000.         3000.         3354.10196625 42

C:\Users\tim\AppData\Local\Temp\ipykernel_42064\4201130403.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  angle[i] = m.atan(opposite/adjacent)


two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [8]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

displacement bcs: [node number, x disp (u), y disp (v)]
[[3 0 0]
 [4 0 None]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
done adjusting displacements array!
u & v list:
[None, None, None, None, 0, 0, 0, None, None, None, None, None, None, None]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [9]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if force_BCs[i,1] or force_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads}")

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print(f"force list: \n{forceList}")

number of nodes with concentrated loads: 4
at nodes: [3, 4, 5, 6]
adjusting array to account for: forces boundary conditions
case 3 for BC 1
Case 2 for BC 1
case 3 for BC 2
Case 2 for BC 2
case 3 for BC 3
Case 2 for BC 3
case 3 for BC 4
Case 2 for BC 4
done adjusting forces array!
force list: 
[[    0.]
 [    0.]
 [    0.]
 [    0.]
 [    0.]
 [    0.]
 [    0.]
 [ 3000.]
 [    0.]
 [-2000.]
 [    0.]
 [ 2000.]
 [    0.]
 [-5000.]]


Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [10]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(f"Global K array dimensions: {globalK.shape}")

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    print(f"E = {E.value}, A ={A.value}, length for element {i+1} = {lengths.value[i]}")
    barConstant[i] = (E.value*A.value)/lengths.value[i]

print(f"Bar Constants = \n{barConstant}")

Global K array dimensions: (14, 14)
E = 210000, A =200, length for element 1 = 4000.0
E = 210000, A =200, length for element 2 = 3000.0
E = 210000, A =200, length for element 3 = 3354.1019662496847
E = 210000, A =200, length for element 4 = 4272.001872658765
E = 210000, A =200, length for element 5 = 3000.0
E = 210000, A =200, length for element 6 = 1802.7756377319945
E = 210000, A =200, length for element 7 = 2500.0
E = 210000, A =200, length for element 8 = 4000.0
E = 210000, A =200, length for element 9 = 4000.0
E = 210000, A =200, length for element 10 = 5656.85424949238
E = 210000, A =200, length for element 11 = 5000.0
Bar Constants = 
[[10500.        ]
 [14000.        ]
 [12521.980674  ]
 [ 9831.45636447]
 [14000.        ]
 [23297.40824146]
 [16800.        ]
 [10500.        ]
 [10500.        ]
 [ 7424.62120246]
 [ 8400.        ]]


## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [11]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print(f"\nk matrix for element {i+1}: \n{kMatrix}")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(f"\nGlobal K matrix: \n{globalK}")
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)


k matrix for element 1: 
[[ 10500.      0. -10500.     -0.]
 [     0.      0.     -0.     -0.]
 [-10500.     -0.  10500.      0.]
 [    -0.     -0.      0.      0.]]
Element 1 starting at node: 1 and ending at node: 2

k matrix for element 2: 
[[ 14000.      0. -14000.     -0.]
 [     0.      0.     -0.     -0.]
 [-14000.     -0.  14000.      0.]
 [    -0.     -0.      0.      0.]]
Element 2 starting at node: 2 and ending at node: 3

k matrix for element 3: 
[[  2504.3961348   5008.7922696  -2504.3961348  -5008.7922696]
 [  5008.7922696  10017.5845392  -5008.7922696 -10017.5845392]
 [ -2504.3961348  -5008.7922696   2504.3961348   5008.7922696]
 [ -5008.7922696 -10017.5845392   5008.7922696  10017.5845392]]
Element 3 starting at node: 3 and ending at node: 7

k matrix for element 4: 
[[ 8619.35900447  3232.25962668 -8619.35900447 -3232.25962668]
 [ 3232.25962668  1212.09736    -3232.25962668 -1212.09736   ]
 [-8619.35900447 -3232.25962668  8619.35900447  3232.25962668]
 [-3232.25962668

In [12]:
j = 0
while j < numNodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    if uvList[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

0
none detected at row 0
1
none detected at row 1
2
none detected at row 2
3
none detected at row 3
4
u = 0 at row: 4
5
u = 0 at row: 5
6
u = 0 at row: 6
7
none detected at row 7
8
none detected at row 8
9
none detected at row 9
10
none detected at row 10
11
none detected at row 11
12
none detected at row 12
13
none detected at row 13
[[ 14212.31060123   3712.31060123 -10500.              0.
       0.              0.             -0.             -0.
   -3712.31060123  -3712.31060123      0.              0.
       0.              0.        ]
 [  3712.31060123  20512.31060123      0.              0.
       0.              0.             -0.         -16800.
   -3712.31060123  -3712.31060123      0.              0.
       0.              0.        ]
 [-10500.              0.          27524.           4032.
  -14000.              0.              0.              0.
      -0.             -0.          -3024.          -4032.
       0.              0.        ]
 [     0.              0.           

In [13]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}\n")
        node += 1 

u1 = [-1.11607143]
v1 = [10.10065492]

u2 = [-0.50892857]
v2 = [3.57375117]

u3 = [-0.]
v3 = [-0.]

u4 = [0.]
v4 = [10.48011921]

u5 = [3.5983349]
v5 = [3.66898927]

u6 = [3.7858349]
v6 = [0.16666667]

u7 = [2.65605978]
v7 = [-1.70237163]



Forces can be found from [K]*[u]

In [14]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


Final Forces: 
[[    0.]
 [    0.]
 [   -0.]
 [   -0.]
 [ 9000.]
 [ 2000.]
 [-9000.]
 [ 3000.]
 [    0.]
 [-2000.]
 [    0.]
 [ 2000.]
 [   -0.]
 [-5000.]]
fx1 = [0.]
fy1 = [0.]
fx2 = [-0.]
fy2 = [-0.]
fx3 = [9000.]
fy3 = [2000.]
fx4 = [-9000.]
fy4 = [3000.]
fx5 = [0.]
fy5 = [-2000.]
fx6 = [0.]
fy6 = [2000.]
fx7 = [-0.]
fy7 = [-5000.]


## Finding Stresses

In [15]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Element 1 starts at node: 1 and ends at node: 2
adjusted start node: 0 adjusted end node: 2
trigMatrix = 
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
local u matrix: 
[[-1.11607143]
 [10.10065492]
 [-0.50892857]
 [ 3.57375117]]
local length matrix: 
[[-0.00025  0.00025]]
Stress in element = [[31.875]]
Element 2 starts at node: 2 and ends at node: 3
adjusted start node: 2 adjusted end node: 4
trigMatrix = 
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
local u matrix: 
[[-0.50892857]
 [ 3.57375117]
 [-0.        ]
 [-0.        ]]
local length matrix: 
[[-0.00033333  0.00033333]]
Stress in element = [[35.625]]
Element 3 starts at node: 3 and ends at node: 7
adjusted start node: 4 adjusted end node: 12
trigMatrix = 
[[0.4472136  0.89442719 0.         0.        ]
 [0.         0.         0.4472136  0.89442719]]
local u matrix: 
[[-0.        ]
 [-0.        ]
 [ 2.65605978]
 [-1.70237163]]
local length matrix: 
[[-0.00029814  0.00029814]]
Stress in element

In [16]:
print(stressFinal)

[[ 31.875     ]
 [ 35.625     ]
 [-20.96313729]
 [ 48.06002107]
 [ 13.125     ]
 [ 11.26734774]
 [ 31.875     ]
 [  5.        ]
 [  8.75      ]
 [-45.0780573 ]
 [ -6.25      ]]


In [17]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)

████████████████████████████████████████████████████████████████████████████████████████████████████
The nodal displacements (u,v), in [mm], ordered from lowest to highest numbered node are:
[[-1.1161]
 [10.1007]
 [-0.5089]
 [3.5738]
 [-0.0000]
 [-0.0000]
 [0.0000]
 [10.4801]
 [3.5983]
 [3.6690]
 [3.7858]
 [0.1667]
 [2.6561]
 [-1.7024]]
████████████████████████████████████████████████████████████████████████████████████████████████████
████████████████████████████████████████████████████████████████████████████████████████████████████

The element axial stresses, in [MPa], ordered from lowest to highest numbered element are:
[[31.8750]
 [35.6250]
 [-20.9631]
 [48.0600]
 [13.1250]
 [11.2673]
 [31.8750]
 [5.0000]
 [8.7500]
 [-45.0781]
 [-6.2500]]

Note: a (+) value indicates the element is in tension, while a (-) value in compression.

████████████████████████████████████████████████████████████████████████████████████████████████████
